In [1]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

# from ner_eval import collect_named_entities
# from ner_eval import compute_metrics
# from ner_eval import compute_precision_recall_wrapper

# from sklearn_crfsuite import metric
from nervaluate import Evaluator

In [2]:
model_name='bert-base-cased'

In [3]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [4]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [8]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [9]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-594-epoch-1',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# # result, model_outputs, wrong_preds = model.eval_model(eval_df)
# # result

In [10]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [11]:
model.train_model(train_df,  eval_data=eval_df)

2021-12-11 19:47:58 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_train_bert_128_13_4752
Epochs 0/5. Running Loss:    0.2445: 100%|██████████| 594/594 [00:49<00:00, 12.04it/s]
2021-12-11 19:48:50 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_128_13_1188
Epochs 1/5. Running Loss:    0.4525: 100%|██████████| 594/594 [00:50<00:00, 11.72it/s]
2021-12-11 19:49:48 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_128_13_1188
Epochs 2/5. Running Loss:    0.0947: 100%|██████████| 594/594 [00:52<00:00, 11.41it/s]
2021-12-11 19:50:48 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_128_13_1188
Epoch 4 of 5:  60%|██████    | 3/5 [02:55<01:58, 59.06s/it]2021-12-11 19:51:15 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_128_13_1188
























Epochs 3/5. Ru

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.24445314705371857,
               0.45251598954200745,
               0.0947263240814209,
               0.030602794140577316,
               0.021818164736032486,
               0.02592386305332184],
              'eval_loss': [0.2541773686272986,
               0.22562414903393727,
               0.22324831808773463,
               0.26333731178181785,
               0.24958776289459644,
               0.2677097524447529],
              'precision': [0.7020089285714286,
               0.7019835510401549,
               0.6963619402985075,
               0.7362581946545638,
               0.7173704414587332,
               0.7332347140039448],
              'recall': [0.5697463768115942,
               0.6571557971014492,
               0.676177536231884,
               0.6612318840579711,
               0.6770833333333334,
               0.6734601449275363],
    

In [12]:
result, model_outputs, y_pred = model.eval_model(test_df)

2021-12-11 19:52:56 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Evaluation: 100%|██████████| 186/186 [00:03<00:00, 61.87it/s]
2021-12-11 19:53:04 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.11604002777609713, 'precision': 0.868440779610195, 'recall': 0.8401015228426396, 'f1_score': 0.854036122373756}


In [13]:
result

{'eval_loss': 0.11604002777609713,
 'precision': 0.868440779610195,
 'recall': 0.8401015228426396,
 'f1_score': 0.854036122373756}

In [14]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [15]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2021-12-11 19:53:04 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 68.96it/s]


In [16]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'O'},
  {'SHOPEE': 'B-PROD'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'},
  {'#AdaMamamoodiShopee': 'O'}]]

In [ ]:
#Now we create the model again using the best estimators

print(sklearn.metrics.classification_report(
    y_test, y_pred, labels=label_list, digits=3
    ))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [17]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [18]:
results, results_per_tag = evaluator.evaluate()

In [19]:
results['partial']

{'correct': 2382,
 'incorrect': 0,
 'partial': 111,
 'missed': 364,
 'spurious': 175,
 'possible': 2857,
 'actual': 2668,
 'precision': 0.9136056971514243,
 'recall': 0.8531676583829192,
 'f1': 0.8823529411764706}

In [20]:
results['exact']

{'correct': 2382,
 'incorrect': 111,
 'partial': 0,
 'missed': 364,
 'spurious': 175,
 'possible': 2857,
 'actual': 2668,
 'precision': 0.8928035982008995,
 'recall': 0.8337416870843543,
 'f1': 0.862262443438914}

In [21]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [22]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [ ]:
import re
for i in range(len(y_test)):
    if len(y_test[i]) != len(y_pred[i]):
        print(x_test[i])
        print(len(x_test[i]))
        print("y_test       ",len(y_test[i]),y_test[i])
        print("y_pred before",len(y_pred[i]),y_pred[i])
        for j in range(len(x_test[i])):
            # if '�' in x_test[i][j]: 
            qmark=re.match(r'^�+$',x_test[i][j])
            if qmark:
                print(qmark)
                y_pred[i].insert(j,'O')
        print("y_pred after ",len(y_pred[i]), y_pred[i])
        print()

['@Yanua02', '@BatuKepla', '@Karsono12732862', '@Andreas50772732', '@AndreMa32495641', '@victorpangkey', '@andriskhas', '@AbyManyuu', '@lizaariani5', '@Nana_Rosaa', '_', '@dwi2dh25', '@BaswirOb', '@Bang__Mus', '@745L023N6', '@Rijani54531379', '@Dewi52502425', '@Celes_being1074', '@Yatimpi97149270', '@begoeloh', '@FEMYLIA_K', '@FKARIN3', '@PRINCESLAMPIR', '@aewin86', '@Elina_Vay', '@amandagempita', '@CynthiaEllenna', '@dchyntiabella', '@Permata_Dewi06', '@Dewianggra91', '@Dewi37492961', '@Putri_Sulita', '@tasya_aa77', '@bunga_lao', '@Rismaliez85', '@SonyaAmriani', '@gituLok', '@hennyindah1404', '@WiwidNurwidaya1', '@OniMeniq74', '@maita_omerta', '@Ek4_Puspita', '@Mahrus30127858', '@Zoer_Riya', '@Reny_soepomo', '@BersamaSahabat4', '@jokowi', '@KPK_RI', '@PerekonomianRI', '@airlangga_hrt', '@KemenkeuRI', 'Infrastruktur', 'adalah', 'fondasi', 'bagi', 'rumah', 'indah', 'Indonesia', 'yg', 'aka', 'dibangun', 'di', 'era', 'pasca', 'Jokowi', 'Jokowi', 'yg', 'bangun', 'fondasi', 'rumahnya', ',',

NameError: name 're' is not defined